In [3]:
import torch
from datasets import load_dataset
from datasets import load_from_disk


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:

# 定义数据集
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, split):
        # self.dataset = load_dataset('lansinuote/ChnSentiCorp', split=split)
        self.dataset = load_from_disk('./data/ChnSentiCorp')['%s' % split]
        print(self.dataset['text'][0])
        print(self.dataset['label'][0])
        # self.dataset = load_from_disk('./data/ChnSentiCorp')
        # print(self.dataset)
        # self.dataset = datasets.fetch_openml('./data/ChnSentiCorp', split=split)
        
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset['text'][i]
        label = self.dataset['label'][i]
        return text, label

In [5]:

dataset = Dataset('train') #['train']

# print(dataset['train'][0:10])
# print(dataset['train'][1])

dataset

选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般
1


In [126]:
dataset[0]

('选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般',
 1)

## 加载模型

In [10]:
from transformers import BertTokenizer
from transformers import BertModel


# 加载字典和分词工具
token = BertTokenizer.from_pretrained('bert-base-chinese')
print('token: ', token)


# 加载预训练模型
pretrained = BertModel.from_pretrained('bert-base-chinese').to(device)
# print('pretrained ', pretrained)

token:  BertTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### 定义 collate_fn 方法

In [11]:
def collate_fn(data):
    # print(type(data))
    # print('data数据: %s' % data)
    # print('data长度: ', len(data))

    
    # 取每个评论的第一个字符作为数据, 第二作为标签
    # 猫和老鼠的DVD,我在当当网已买过10余次了。除了做为礼物送给亲朋好有的孩子外，...
    
    sents =  [i[0] for i in data] # 猫 ...  16个
    labels = [i[1] for i in data] # 和 ...   16个
    
    #print('sents: %s sents长度: %s' % (sents, len(sents)))
    #print('labels: %s' % labels)    
    
    # 编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=500,
                                   return_tensors='pt',
                                   return_length=True
                                  )
    
    # dict_keys(['input_ids', 'token_type_ids', 'length', 'attention_mask'])
    # print('编码后的date: ', data.keys()) 
    
    #print('input_ids状态: ', data['input_ids'].shape) [16, 500]
    #print('token_type_ids状态: ', data['token_type_ids'].shape) [16, 500]
    #print('attention_mask状态: ', data['attention_mask'].shape) [16, 500]
    
    # 打印
    # print(data['input_ids'])
    
    
    # input_ids:编码之后的数字
    # attention_mask:是补零的位置是0,其他位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']

    return input_ids, attention_mask, token_type_ids, labels

## 数据加载器

In [12]:

# 数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)


for i, values in enumerate(loader):
    input_ids, attention_mask, token_type_ids, labels = values
    
    print('')
    # print(i, values)
    print(i, input_ids.shape, 
          attention_mask.shape, 
          token_type_ids.shape, 
          labels
         )
    print('')
    if i >= 2:
        break
    

print(len(loader))


0 torch.Size([16, 500]) torch.Size([16, 500]) torch.Size([16, 500]) [0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0]


1 torch.Size([16, 500]) torch.Size([16, 500]) torch.Size([16, 500]) [0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0]


2 torch.Size([16, 500]) torch.Size([16, 500]) torch.Size([16, 500]) [1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0]

600


## 模型训练

In [13]:
# 不训练,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)

    
#模型试算
#out = pretrained(input_ids=input_ids,
#           attention_mask=attention_mask,
#           token_type_ids=token_type_ids)

#out.last_hidden_state.shape

In [14]:
# 定义下游任务模型
class Model(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                             attention_mask=attention_mask,
                             token_type_ids=token_type_ids)
            pass
        out = self.fc(out.last_hidden_state[:, 0])
        out = out.softmax(dim=1)
        return out


model = Model()


#model(input_ids=input_ids,
#      attention_mask=attention_mask,
#      token_type_ids=token_type_ids
#     ).shape


model.to(device)

Model(
  (fc): Linear(in_features=768, out_features=2, bias=True)
)

## 模型训练

In [158]:
from transformers import AdamW


# 训练
optimizer = AdamW(model.parameters(), lr=5e-4)
loss = torch.nn.CrossEntropyLoss()


model.train()


for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
    
    # print('input_ids.shape: %s, labels.shape: %s \n' % (input_ids.shape, labels.shape))
    # print('input_ids: %s, labels: %s' % (input_ids, labels))
    # print("")
    
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    token_type_ids = token_type_ids.to(device)
    labels = torch.tensor(labels).to(device)
        
    out = model(input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
    # print('')
    # print('out状态: ', out, out.shape)
    # print('labels状态: ', labels, labels)
    
    # 梯度下降
    l = loss(out, labels)
    
    optimizer.zero_grad()
    l.backward()
    optimizer.step()
    

    if i % 5 == 0:
        out = out.cpu()
        labels = labels.cpu()
            
        out = out.argmax(dim=1)
        accuracy = (out == labels).sum().item() / len(labels)
        print(i, l.item(), accuracy)

    if i == 300:
        break

0 0.6958725452423096 0.625
5 0.7487972378730774 0.4375
10 0.6864623427391052 0.5625
15 0.7131083607673645 0.4375
20 0.5900887846946716 0.75
25 0.5897844433784485 0.875
30 0.6619393229484558 0.5625
35 0.6275553107261658 0.625
40 0.564215362071991 0.875
45 0.5329274535179138 0.9375
50 0.5611305832862854 0.875
55 0.5218709111213684 0.9375
60 0.5288110971450806 0.9375
65 0.5353909134864807 0.875
70 0.5484877228736877 0.8125
75 0.5157537460327148 0.875
80 0.4717966914176941 0.9375
85 0.5041393041610718 0.8125
90 0.4816528558731079 0.9375
95 0.47506433725357056 0.875
100 0.47736144065856934 0.9375
105 0.42665037512779236 1.0
110 0.49181699752807617 0.8125
115 0.46781566739082336 0.875
120 0.4107031226158142 0.9375
125 0.44184234738349915 0.9375
130 0.4267587661743164 0.875
135 0.4708537757396698 0.875
140 0.45660653710365295 0.875
145 0.5373218059539795 0.8125
150 0.44355103373527527 0.9375
155 0.4072643220424652 0.9375
160 0.4500364661216736 0.9375
165 0.5201813578605652 0.875
170 0.4712945

## 测试

In [15]:

def test_res():
    model.eval()
    correct = 0
    total = 0

    loader_test = torch.utils.data.DataLoader(dataset=Dataset('validation'),
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader_test):
        
        if i > 100:
            break
            
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        labels = torch.tensor(labels).to(device)

        with torch.no_grad():
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)
            pass
        
        out = out.cpu()
        labels = labels.cpu()
        out = out.argmax(dim=1)
        
        # print('text: %s, 预测值: %s, 真实值: %s' % (input_ids, out, labels))
        
        correct += (out == labels).sum().item()
        total += len(labels)
        
        print('当前批次, acc: %s' % (out.sum() / (out == labels).sum()))

    print('acc: ', correct / total)


In [16]:

# 测试结果
test_res()

這間酒店環境和服務態度亦算不錯,但房間空間太小~~不宣容納太大件行李~~且房間格調還可以~~ 中餐廳的廣東點心不太好吃~~要改善之~~~~但算價錢平宜~~可接受~~ 西餐廳格調都很好~~但吃的味道一般且令人等得太耐了~~要改善之~~
1
当前批次, acc: tensor(0.8636)
当前批次, acc: tensor(1.3125)
当前批次, acc: tensor(0.6400)
当前批次, acc: tensor(0.7917)
当前批次, acc: tensor(0.8000)
当前批次, acc: tensor(0.9130)


KeyboardInterrupt: 

In [235]:
dataset

In [17]:


str_v1 = '这句话的意思是，”因为我在思考，所以我存在”。笛卡尔把这句话作为他哲学思考的起点，他试图通过怀疑一切，最终找到一个不容置疑的真理。'


out = token.encode_plus(str_v1,
                        # 当句子长度大于max_length时,截断
                        truncation=True,
                        # 一律补pad到 max_length长度
                        padding='max_length',
                        add_special_tokens=True,
                        max_length=500,

                        #可取值tf,pt,np,默认为返回list
                        return_tensors=None,
                        #返回token_type_ids
                        return_token_type_ids=True,
                        #返回attention_mask
                        return_attention_mask=True,
                        #返回special_tokens_mask 特殊符号标识
                        return_special_tokens_mask=True,

                        #返回offset_mapping 标识每个词的起止位置,
                        # 这个参数只能BertTokenizerFast使用
                        #return_offsets_mapping=True,

                        #返回length 标识长度
                        return_length=True
                       )

In [256]:
# out

In [19]:

input_ids = torch.tensor([out['input_ids']]).to(device)
attention_mask = torch.tensor([out['attention_mask']]).to(device)
token_type_ids = torch.tensor([out['token_type_ids']]).to(device)


out_test = model(input_ids=input_ids,
                 attention_mask=attention_mask,                 
                 token_type_ids=token_type_ids
                )


out_res = out_test.argmax(dim=1).cpu()


print(out_res)

tensor([1])
